In [35]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
tokens = tokenizer("On y va.", return_offsets_mapping=True)
tokens.offset_mapping

loading configuration file https://huggingface.co/nlpaueb/legal-bert-base-uncased/resolve/main/config.json from cache at /Users/ellabettison/.cache/huggingface/transformers/8fb343045f345372593fb03cc8edd339757b9b5fa164bfd4a071c33d81906423.81e683d6e4f3240a109c9cfd0742b01d443acc6fc2b120b20796fb85ddf0f393
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size

[(0, 0), (0, 2), (3, 4), (5, 7), (7, 8), (0, 0)]

In [36]:
tags = [
  "ID",
  "PER",
  "IND",
  "STP",
  "LOC",
  "ORG",
  "NAT",
  "COL",
  "LEG",
  "ACR",
  "STR",
  "SCP",
  "MOD"
]
id2tag = {i+1:v for i, v in enumerate(tags)}
tag2id = {v:i+1 for i, v in enumerate(tags)}
id2tag

{1: 'ID',
 2: 'PER',
 3: 'IND',
 4: 'STP',
 5: 'LOC',
 6: 'ORG',
 7: 'NAT',
 8: 'COL',
 9: 'LEG',
 10: 'ACR',
 11: 'STR',
 12: 'SCP',
 13: 'MOD'}

In [37]:
label2id = {
    'O': 0, 
    **{f'B-{k}': 2*v - 1 for k, v in tag2id.items()},
    **{f'I-{k}': 2*v for k, v in tag2id.items()}
}

id2label = {v:k for k, v in label2id.items()}

id2label

{0: 'O',
 1: 'B-ID',
 3: 'B-PER',
 5: 'B-IND',
 7: 'B-STP',
 9: 'B-LOC',
 11: 'B-ORG',
 13: 'B-NAT',
 15: 'B-COL',
 17: 'B-LEG',
 19: 'B-ACR',
 21: 'B-STR',
 23: 'B-SCP',
 25: 'B-MOD',
 2: 'I-ID',
 4: 'I-PER',
 6: 'I-IND',
 8: 'I-STP',
 10: 'I-LOC',
 12: 'I-ORG',
 14: 'I-NAT',
 16: 'I-COL',
 18: 'I-LEG',
 20: 'I-ACR',
 22: 'I-STR',
 24: 'I-SCP',
 26: 'I-MOD'}

In [38]:
def get_token_role_in_span(token_start: int, token_end: int, span_start: int, span_end: int):
    """
    Check if the token is inside a span.
    Args:
      - token_start, token_end: Start and end offset of the token
      - span_start, span_end: Start and end of the span
    Returns:
      - "B" if beginning
      - "I" if inner
      - "O" if outer
      - "N" if not valid token (like <SEP>, <CLS>, <UNK>)
    """
    if token_end <= token_start:
        return "N"
    if token_start < span_start or token_end > span_end:
        return "O"
    if token_start > span_start:
        return "I"
    else:
        return "B"

MAX_LENGTH = 16

def tokenize_and_adjust_labels(sample):
    """
    Args:
        - sample (dict): {"id": "...", "text": "...", "tags": [{"start": ..., "end": ..., "tag": ...}, ...]
    Returns:
        - The tokenized version of `sample` and the labels of each token.
    """
    # Tokenize the text, keep the start and end positions of tokens with `return_offsets_mapping` option
    # Use max_length and truncation to ajust the text length
    tokenized = tokenizer(sample["text"], 
                          return_offsets_mapping=True, 
                          padding="max_length", 
                          max_length=MAX_LENGTH,
                          truncation=True)
    
    # We are doing a multilabel classification task at each token, we create a list of size len(label2id)=13 
    # for the 13 labels
    labels = [[0 for _ in label2id.keys()] for _ in range(MAX_LENGTH)]
    
    # Scan all the tokens and spans, assign 1 to the corresponding label if the token lies at the beginning
    # or inside the spans
    for (token_start, token_end), token_labels in zip(tokenized["offset_mapping"], labels):
        for span in sample["tags"]:
            role = get_token_role_in_span(token_start, token_end, span["start"], span["end"])
            if role == "B":
                token_labels[label2id[f"B-{span['tag']}"]] = 1
            elif role == "I":
                token_labels[label2id[f"I-{span['tag']}"]] = 1
    
    return {**tokenized, "labels": labels}

In [39]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
!pip install spacy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
import json

def spacy_to_jsonl(spacy_eg, filename):
    json_ls = []

    for i, sent in enumerate(spacy_eg):
        json_l = {}
        tags = []
        for token in sent.spans['sc']:
            tags.append({'end':token.end_char, 'start':token.start_char, 'tag':token.label_})
        json_l['tags'] = tags
        json_l['id'] = i
        json_l['text'] = sent.text

        json_ls.append(json_l)
    with open(filename, 'w') as fp:
        fp.write(
            '[' +
            ',\n'.join(json.dumps(i) for i in json_ls) +
        ']\n')
    

In [42]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load('output/model-best')

def load_spacy(input_path:str) -> list:
    doc_bin = DocBin().from_disk(input_path)
    return (list(doc_bin.get_docs(nlp.vocab)))

test_examples = load_spacy("../spacy_data/dev.spacy")
spacy_to_jsonl(test_examples, "test_data.jsonl")

test_examples = load_spacy("../spacy_data/train.spacy")
spacy_to_jsonl(test_examples, "train_data.jsonl")





loading configuration file /var/folders/yg/f3nqzkxd36d391q7jr9jxtv00000gn/T/tmpea1ai0ca/config.json
Model config RobertaConfig {
  "_name_or_path": "/var/folders/yg/f3nqzkxd36d391q7jr9jxtv00000gn/T/tmpea1ai0ca/config.json",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

Didn't find file /var/folders/yg/f3nqzkxd36d391q7jr9jxtv00000gn/T/tmpea1ai0ca/added_tokens.json. We won't load it.
loading file /var/folders/yg/f3nqzkxd36d391q7jr9jxtv0

In [43]:
from datasets import Dataset
train_ds = Dataset.from_json("test_data.jsonl")
val_ds = Dataset.from_json("train_data.jsonl")

train_ds[0]

Using custom data configuration default-c89e8d4d06f7b4f5




Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 132.94it/s]


Using custom data configuration default-0f492f6c38e0465b


Dataset json downloaded and prepared to /Users/ellabettison/.cache/huggingface/datasets/json/default-c89e8d4d06f7b4f5/0.0.0. Subsequent calls will reuse this data.




Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 236.23it/s]




Dataset json downloaded and prepared to /Users/ellabettison/.cache/huggingface/datasets/json/default-0f492f6c38e0465b/0.0.0. Subsequent calls will reuse this data.


{'tags': [{'end': 2, 'start': 0, 'tag': 'ID'},
  {'end': 2, 'start': 0, 'tag': 'ACR'}],
 'id': 0,
 'text': 'CA'}

In [44]:
for i in range(3):
    example = train_ds[i]
    print(f"\n{example['text']}")
    for tag_item in example["tags"]:
        print(tag_item["tag"].ljust(10), "-", example["text"][tag_item["start"]: tag_item["end"]])


CA
ID         - CA
ACR        - CA

CDCA
ACR        - CDCA
ID         - CDCA

bottom line training and consulting
ID         - bottom line
IND        - training
STP        - and
IND        - consulting


In [45]:
tokenized_train_ds = train_ds.map(tokenize_and_adjust_labels, remove_columns=train_ds.column_names)
tokenized_val_ds = val_ds.map(tokenize_and_adjust_labels, remove_columns=val_ds.column_names)



100%|██████████| 234/234 [00:00<00:00, 1961.40ex/s]




100%|██████████| 938/938 [00:00<00:00, 2836.04ex/s]


In [46]:
sample = tokenized_train_ds[0]
sample["labels"]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [47]:
print(tokenized_train_ds[0])

{'input_ids': [101, 2121, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'offset_mapping': [[0, 0], [0, 2], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]], 'labels': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [48]:
%pip install sklearn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [49]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

n_labels = len(id2label)

def divide(a: int, b: int):
    return a / b if b > 0 else 0

def compute_metrics(p):
    """
    Customize the `compute_metrics` of `transformers`
    Args:
        - p (tuple):      2 numpy arrays: predictions and true_labels
    Returns:
        - metrics (dict): f1 score on 
    """
    # (1)
    predictions, true_labels = p
    
    # (2)
    predicted_labels = np.where(predictions > 0, np.ones(predictions.shape), np.zeros(predictions.shape))
    metrics = {}
    
    # (3)
    cm = multilabel_confusion_matrix(true_labels.reshape(-1, n_labels), predicted_labels.reshape(-1, n_labels))
    
    # (4) 
    for label_idx, matrix in enumerate(cm):
        if label_idx == 0:
            continue # We don't care about the label "O"
        tp, fp, fn = matrix[1, 1], matrix[0, 1], matrix[1, 0]
        precision = divide(tp, tp + fp)
        recall = divide(tp, tp + fn)
        f1 = divide(2 * precision * recall, precision + recall)
        metrics[f"f1_{id2label[label_idx]}"] = f1
        
    # (5)
    macro_f1 = sum(list(metrics.values())) / (n_labels - 1)
    metrics["macro_f1"] = macro_f1
        
    return metrics

In [50]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import RobertaPreTrainedModel, RobertaModel
from transformers.utils import (
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.models.roberta.modeling_roberta import (
    ROBERTA_INPUTS_DOCSTRING,
    ROBERTA_START_DOCSTRING,
    RobertaEmbeddings,
    _TOKENIZER_FOR_DOC
)
from typing import Optional, Union, Tuple
from transformers.modeling_outputs import TokenClassifierOutput
import torch
from torch import nn

class RobertaForSpanCategorization(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]
    _keys_to_ignore_on_load_missing = [r"position_ids"]
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Initialize weights and apply final processing
        self.post_init()
    
    @add_start_docstrings_to_model_forward(ROBERTA_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [54]:
training_args = TrainingArguments(
    output_dir="./models/fine_tune_bert_output_span_cat",
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=2.5e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps = 10,
    save_strategy='steps',
    save_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    seed=12345
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [55]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [56]:
def model_init():
    # For reproducibility
    return RobertaForSpanCategorization.from_pretrained("nlpaueb/legal-bert-base-uncased", id2label=id2label, label2id=label2id)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

loading configuration file https://huggingface.co/nlpaueb/legal-bert-base-uncased/resolve/main/config.json from cache at /Users/ellabettison/.cache/huggingface/transformers/8fb343045f345372593fb03cc8edd339757b9b5fa164bfd4a071c33d81906423.81e683d6e4f3240a109c9cfd0742b01d443acc6fc2b120b20796fb85ddf0f393
You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Model config RobertaConfig {
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-ID",
    "2": "I-ID",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-IND",
    "6": "I-IND",
    "7": "B-STP",
    "8": "I-STP",
    "9": "B-LOC",
    "10": "I-LOC",
    "11": "B-ORG",
    "12": "I-ORG",
    "13":

{'loss': 0.1544, 'learning_rate': 0.0002483333333333333, 'epoch': 0.67}




















































































































































































                                           


                                                   

                                            



                                   

  0%|          | 1/1500 [04:09<14:08:39, 33.97s/it]






Saving model checkpoint to ./models/fine_tune_bert_output_span_cat/checkpoint-10
Configuration saved in ./models/fine_tune_bert_output_span_cat/checkpoint-10/config.json


{'eval_loss': 0.06250406056642532, 'eval_f1_B-ID': 0, 'eval_f1_I-ID': 0, 'eval_f1_B-PER': 0, 'eval_f1_I-PER': 0, 'eval_f1_B-IND': 0, 'eval_f1_I-IND': 0, 'eval_f1_B-STP': 0, 'eval_f1_I-STP': 0, 'eval_f1_B-LOC': 0, 'eval_f1_I-LOC': 0, 'eval_f1_B-ORG': 0, 'eval_f1_I-ORG': 0, 'eval_f1_B-NAT': 0, 'eval_f1_I-NAT': 0, 'eval_f1_B-COL': 0, 'eval_f1_I-COL': 0, 'eval_f1_B-LEG': 0, 'eval_f1_I-LEG': 0, 'eval_f1_B-ACR': 0, 'eval_f1_I-ACR': 0, 'eval_f1_B-STR': 0, 'eval_f1_I-STR': 0, 'eval_f1_B-SCP': 0, 'eval_f1_I-SCP': 0, 'eval_f1_B-MOD': 0, 'eval_f1_I-MOD': 0, 'eval_macro_f1': 0.0, 'eval_runtime': 35.6809, 'eval_samples_per_second': 26.289, 'eval_steps_per_second': 1.654, 'epoch': 0.67}


Model weights saved in ./models/fine_tune_bert_output_span_cat/checkpoint-10/pytorch_model.bin
tokenizer config file saved in ./models/fine_tune_bert_output_span_cat/checkpoint-10/tokenizer_config.json
Special tokens file saved in ./models/fine_tune_bert_output_span_cat/checkpoint-10/special_tokens_map.json





















                                           


                                                   

                                            

  0%|          | 1/1500 [04:34<14:08:39, 33.97s/it]
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSpanCategorization.forward` and have been ignored: offset_mapping. If offset_mapping are not expected by `RobertaForSpanCategorization.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 938
  Batch size = 16


...

{'loss': 0.0549, 'learning_rate': 0.00024166666666666667, 'epoch': 3.33}




















































































































































































                                           


                                                   

                                            



                                   

  0%|          | 1/1500 [08:17<14:08:39, 33.97s/it]






Saving model checkpoint to ./models/fine_tune_bert_output_span_cat/checkpoint-50
Configuration saved in ./models/fine_tune_bert_output_span_cat/checkpoint-50/config.json


{'eval_loss': 0.053401727229356766, 'eval_f1_B-ID': 0, 'eval_f1_I-ID': 0, 'eval_f1_B-PER': 0, 'eval_f1_I-PER': 0, 'eval_f1_B-IND': 0, 'eval_f1_I-IND': 0, 'eval_f1_B-STP': 0, 'eval_f1_I-STP': 0, 'eval_f1_B-LOC': 0, 'eval_f1_I-LOC': 0, 'eval_f1_B-ORG': 0, 'eval_f1_I-ORG': 0, 'eval_f1_B-NAT': 0, 'eval_f1_I-NAT': 0, 'eval_f1_B-COL': 0, 'eval_f1_I-COL': 0, 'eval_f1_B-LEG': 0, 'eval_f1_I-LEG': 0, 'eval_f1_B-ACR': 0, 'eval_f1_I-ACR': 0, 'eval_f1_B-STR': 0, 'eval_f1_I-STR': 0, 'eval_f1_B-SCP': 0, 'eval_f1_I-SCP': 0, 'eval_f1_B-MOD': 0, 'eval_f1_I-MOD': 0, 'eval_macro_f1': 0.0, 'eval_runtime': 35.9208, 'eval_samples_per_second': 26.113, 'eval_steps_per_second': 1.643, 'epoch': 3.33}


Model weights saved in ./models/fine_tune_bert_output_span_cat/checkpoint-50/pytorch_model.bin
tokenizer config file saved in ./models/fine_tune_bert_output_span_cat/checkpoint-50/tokenizer_config.json
Special tokens file saved in ./models/fine_tune_bert_output_span_cat/checkpoint-50/special_tokens_map.json
Deleting older checkpoint [models/fine_tune_bert_output_span_cat/checkpoint-40] due to args.save_total_limit












KeyboardInterrupt: 